Importing CKAN Datasets + Resources from a CSV
=========================

What seems like a long overdue tutorial for frustrated dataset maintainers who are nearly in tears that it took months to set up their CKAN and now have only a few days to upload a vast amount of data.
--------------------------------------


This notebook describes the process we used to bulk load new datasets into a CKAN data portal from a spreadsheet/csv.

CKAN's documentation for bulk loading / batch import is sparse, [outdated](http://docs.ckan.org/en/ckan-1.7.4/loading-data.html) (aimed at CKAN 1.7 vs 2.\*), or organized around using tools like the Harvester plugin which is aimed at loading from pre-existing feeds in other places. There's a lovely [ckan-csv-mass-import](https://github.com/nucleo-digital/ckan-csv-mass-import) node script, but it doesn't connect new resources to datasets (the files that people want to see). 

Most normies with a bunch of data files just want to put it in a spreadsheet somewhere and run an ETL script so it magically shows up on your data portal. Meanwhile all the documentation keeps talking about these _[datapackage.json](http://frictionlessdata.io/guides/data-package/) things_ that you've never seen or heard of before. This helps with that. Sort of.

Luckily the fine people of Canada have blessed us with the [ckanapi](https://github.com/ckan/ckanapi) command line utility / python module. Once you have a working CKAN instance (which is no small feat), it's a minor miracle for keeping the system up and running. The CKANAPI tool makes loading new datasets & resources possible in the form of datapackages.

This notebook:
1. Takes a CSV based on a specific template describing Datasets and Resources (and the organizations and groups they belong to, but only barely)
2. Connects to your CKAN to ensure that all groups and organizations in your spreadsheet also exist on the site database, and creates any missing ones.
3. Creates an output file of all the datasets to be imported (or reimported if the names are already identical to existing CKAN things)
4. Uses the `ckanapi` command line tool to upload the new datasets + resources to your CKAN site

This process assumes:
* You have a working CKAN (I've tested against 2.6.1. but your experience may vary. Our setup is [documented here](https://github.com/browninstitute/ckan-dockercompose-nuevanacion-tabulario)).
* CKAN Datastore and Datapusher plugins are active and working. Datastore uses Postgres to store & serve structured data to the data preview system. Datapusher is a plugin that does the work of asynchronously loading that data.
* You have an admin account on that CKAN and an API key for that account
* Your files are available on a publically available webserver. We use Amazon S3. No file uploads directly into CKAN.
* You have SSH access to your CKAN instance so you can...
 * you have the ability to run [paster commands](http://docs.ckan.org/en/ckan-2.6.2/maintaining/paster.html) on your CKAN. **try running some commands with `paster` before starting this, otherwise you might wind up with a bunch of datasets missing views or missing from the search index so yeah thats not a good thing**
* You have a high tolerance for long, rambly, often vague documentation. You _ARE_ using CKAN and got this far so...


# A Warning

![animated gif of a cat jumping up behind a toddler and knocking them playfully into the snow to serve as a warning](https://media.giphy.com/media/uTCAwWNtz7U2c/giphy.gif?response_id=59244d4d3a961a1c550f4959)

**CKAN documentation has the tendency to be long, tough to penetrate, covers the bare minimum of use cases, and goes out of date rapidly. This is no exception.**

Creating descriptive metadata is deceptively challenging. As much as you and I want magic script that magically makes datasets appear on the internet exactly as imagined, flat files are a poor way to describe the formatting required for these resources (which is why the [datapackages.json format](http://frictionlessdata.io/guides/data-package/) exists). The CKAN web interface has a metadata authoring workflow that does a good job at helping you write descriptive metadata that'll look good in CKAN and be discoverable by your users. It's a harder to ensure that you'll describe things right here. And even if you do, things are likely to break if you use this.

Please don't hate me if this data import screws up your CKAN.

It's a very good idea to take a database backup before you do this. And also make sure you can actually restore that database backup.

**But seriously this is risky. Please don't trust that it will work. It is not worthy of your trust. It doesn't even completely work for me yet.**


# Lingo
CKAN has its own words. They have [their own documentation for those words](http://docs.ckan.org/en/latest/user-guide.html#datasets-and-resources). This is a bad attempt at reiterating that.

* Dataset: A dataset is the primary catalog record for describing stuff. It consists of a name (required), freeform notes description (optional but really the most important part of making your data usable), a list of tags (optional, but extraordinarially useful) a data license (if you don't license your data, its not open), a list of Resources (described below). Datasets MUST belong to a single Organization (required, described below, and must be created first). Datasets MAY belong to multiple groups (optional, described below, and must be created first, and this script really only works for one group so I'm kind of a jerk for teasing that it might be possible here). 
* Resource: A dataset MAY contain resources. These are files or links that users can get to to directly retrieve the data they're looking for. A dataset may consist of a URL/File, a name, a filetype (e.g. CSV, PDF), and a text description. If a user searches for something that's in a Resource description, their search results will go to the Dataset that contains it. Resources may be deliberately ordered. A Resource MUST be attached to a Dataset to be discoverable to an end-user.

* Organization: The REQUIRED hierarchy that a Dataset MUST belong to ONLY one of. Describes who publishes the data. Maybe its you, maybe its the originator. Figure it out for your use case. Has a name (required), a description (optional - and if created using this script you'll have to add it via the web interface), an image (optional - and if created using this script you'll have to add it via the web interface). It can theoretically be in charge of users or permissions but we're not touching that here.
* Group: The Optional hierarchy that a dataset MAY belong to MULTIPLE of. Has a name (required), a description (optional - and if created using this script you'll have to add it via the web interface), an image (optional - and if created using this script you'll have to add it via the web interface). It can theoretically be in charge of users or permissions but we're not touching that here.

There's other stuff you need to know, like each of these things gets a URL slug (`the-thing-that-looks-like-this-in-a-url`) and those have to be unique, not have spaces, be in only lowercase ASCII letters or dashes/underlines, not have parentheses, etc. But you know whatever you're a smart person I'm sure you'll figure it out. (Have I discouraged you from using this yet? Perhaps I can discourage you from using this! Or perhaps the team working on CKAN can make this significantly clearer because that'd be great!)




# Install prerequisites
You're already doing this in an iPython notebook so I'm assuming you have that but `¯\_(ツ)_/¯`

`! pip install unidecode numpy pandas ckanapi "jupyter [notebook]"`

In [107]:
from ckanapi import RemoteCKAN
import pandas as pd
import numpy as np
import unidecode
import json
import re
from collections import OrderedDict

# What should my CSV look like?
Funny you should ask!

In [3]:
pd.read_csv('full-catalog.csv', header=[0,1])

NOT IMPORTED        DATASET  \
                                         Path/File Name Group (Topics)   
0                                                   NaN  Data Bancaria   
1                                                   NaN  Data Bancaria   
2                                                   NaN  Data Bancaria   
3                                                   NaN  Data Bancaria   
4                                                   NaN  Data Bancaria   
5                                                   NaN  Data Bancaria   
6                                                   NaN  Data Bancaria   
7                                                   NaN  Data Bancaria   
8                                                   NaN  Data Bancaria   
9                                                   NaN  Data Bancaria   
10                                                  NaN  Data Bancaria   
11                                                  NaN  Data Bancaria   
12                                                  NaN  Data Bancaria   
13                                                  NaN  Data Bancaria   
14                                                  NaN  Data Bancaria   
15                                                  NaN  Data Bancaria   
16                                                  NaN  Data Bancaria   
17                                                  NaN  Data Bancaria   
18                                                  NaN  Data Bancaria   
19                                                  NaN  Data Bancaria   
20                                                  NaN  Data Bancaria   
21                                                  NaN       Planilla   
22                                                  NaN       Planilla   
23                                                  NaN       Planilla   
24                                                  NaN       Planilla   
25                                                  NaN       Planilla   
26                                                  NaN       Planilla   
27                                                  NaN       Planilla   
28                                                  NaN       Planilla   
29                                                  NaN       Planilla   
...                                                 ...            ...   
1602  contraloria_gob_pa_tribunal_de_cuentas_2017_01...       Planilla   
1603  contraloria_gob_pa_tribunal_de_cuentas_2017_03...       Planilla   
1604  contraloria_gob_pa_tribunal_de_cuentas_2017_04...       Planilla   
1605  contraloria_gob_pa_tribunal_electoral_2015_12_...       Planilla   
1606  contraloria_gob_pa_tribunal_electoral_2015_12_...       Planilla   
1607  contraloria_gob_pa_tribunal_electoral_2016_01_...       Planilla   
1608  contraloria_gob_pa_tribunal_electoral_2016_02_...       Planilla   
1609  contraloria_gob_pa_tribunal_electoral_2016_02_...       Planilla   
1610  contraloria_gob_pa_tribunal_electoral_2016_03_...       Planilla   
1611  contraloria_gob_pa_tribunal_electoral_2016_04_...       Planilla   
1612  contraloria_gob_pa_tribunal_electoral_2016_05_...       Planilla   
1613  contraloria_gob_pa_tribunal_electoral_2016_06_...       Planilla   
1614  contraloria_gob_pa_tribunal_electoral_2016_06_...       Planilla   
1615  contraloria_gob_pa_tribunal_electoral_2016_06_...       Planilla   
1616  contraloria_gob_pa_tribunal_electoral_2016_07_...       Planilla   
1617  contraloria_gob_pa_tribunal_electoral_2016_07_...       Planilla   
1618  contraloria_gob_pa_tribunal_electoral_2016_08_...       Planilla   
1619  contraloria_gob_pa_tribunal_electoral_2016_08_...       Planilla   
1620  contraloria_gob_pa_tribunal_electoral_2016_09_...       Planilla   
1621  contraloria_gob_pa_tribunal_electoral_2016_09_...       Planilla   
1622  contraloria_gob_pa_tribunal_electoral_2016_10_...       Planilla   
1623  contraloria_gob_pa_tribunal_electoral_201

Columns are split between 2 groups:
1. DATASET: Attributes that apply to the whole Dataset
2. RESOURCE: Individual resources that are attached to a Dataset

To attach multiple Resources to the same Dataset, all the Dataset fields should be IDENTICAL and MUST follow one another. Otherwise it won't pick up on the fact that the resource is part of the same Dataset. The order of Resources is determined by their order in this spreadsheet.

//TODO: Other things folks should keep in mind


# Basic Overview of How This Works:
1. Connects to your CKAN instance
2. Downloads the list of all the groups + organizations
3. Opens the CSV of assets you want to add
4. Tests to see if all the Groups in the CSV are also in the site (determined by an identical display name). If any are missing, they are created
5. Tests to see if all the Organizations in the CSV are also in the site (determined by an identical display name). if any are missing, they are created
6. Creates a minimalist (possibly non-standards-compliant) `datapackage.json` for each Dataset in your spreadsheet comprised of each of your dataset links. Each Dataset is assigned a [Creative Commons CC-Zero Public Domain Dedication](https://creativecommons.org/publicdomain/zero/1.0/) because software is supposed to be opinionated.
7. Outputs a single file with each datapackage JSON object as a line in a text file (aka [jsonlines](http://jsonlines.org/))
8. Calls the `ckanapi` command line utility to actually upload/update the resources. If there are any errors, they'll be output there and you can run this notebook again after you've made changes (either to your CSV or to the parser here) to fix them. Hopefully.




# DANGER ZONE
![Animated GIF of a klaxon and siren going off behind Larry Wilmore on the Larry Wilmore show indicating danger or emergency](https://media.giphy.com/media/ic8d9yo1E3ar6/giphy.gif?response_id=592463e28e7faf33f0f76d33)

**Beginning here, you have the potential to begin making changes to your production CKAN data, which could ruin everything you've done so far. Now is the time to take a database backup. Or several. And test to make sure you can restore it. Please.**

# CKAN Configuration: CHANGE THESE
Enter required fields to talk to your CKAN:
* The CKAN URL + an API Key with sysadmin rights.
* The location of your CSV

In [108]:
CKAN_URL = 'http://data.nuevanacion.com'
CKAN_APIKEY = '' # Your API Key Goes Here
CATALOG_CSV = 'full-catalog.csv'
OUTPUT_DATASETS_JSONL = 'importable-datasets.jsonl'

ckan = RemoteCKAN(address=CKAN_URL, apikey=CKAN_APIKEY)

Gets all Groups and Organizations from your server

In [109]:
groups = ckan.action.group_list(all_fields=True, limit=100000)
groups_nameindex = {}
for g in groups:
    groups_nameindex[g['display_name']] = g
def reindex_groups():
    global groups
    global groups_nameindex
    groups = ckan.action.group_list(all_fields=True, limit=100000)
    for g in groups:
        groups_nameindex[g['display_name']] = g
    return (groups, groups_nameindex)

In [110]:
orgs = ckan.action.organization_list(all_fields=True, limit=100000)
orgs_nameindex = {}
for org in orgs:
    orgs_nameindex[org['display_name']] = org
def reindex_orgs():
    global orgs
    global orgs_nameindex
    orgs = ckan.action.organization_list(all_fields=True, limit=100000)
    for org in orgs:
        orgs_nameindex[org['display_name']] = org

Load the catalog CSV

In [111]:
catalog = pd.read_csv(CATALOG_CSV, skip_blank_lines=True, skiprows=1) # Template CSV Skips first header row.
catalog = catalog.replace(np.nan, '', regex=True) #Blank text is treated as a Numpy Not a Number (NaN). Here we replace that with blank text again.

In [112]:
catalog

,Group (Topics),Organization,Tags (comma-separated),Dataset Name,Dataset Description,Resource Name,Resource Description,Resource URL
0,Data Bancaria,Superintendencia de Bancos,"csv, feci, data bancaria",Reportes de FECI,Incluye todos los reportes del FECI en formato...,FECI 2002,Reporte del FECI para el 2002,http://tabulario.s3.amazonaws.com/Data/Superin...
1,Data Bancaria,Superintendencia de Bancos,"csv, feci, data bancaria",Reportes de FECI,Incluye todos los reportes del FECI en formato...,FECI 2003,Reporte del FECI para el 2003,http://tabulario.s3.amazonaws.com/Data/Superin...
2,Data Bancaria,Superintendencia de Bancos,"csv, feci, data bancaria",Reportes de FECI,Incluye todos los reportes del FECI en formato...,FECI 2004,Reporte del FECI para el 2004,http://tabulario.s3.amazonaws.com/Data/Superin...
3,Data Bancaria,Superintendencia de Bancos,"csv, feci, data bancaria",Reportes de FECI,Incluye todos los reportes del FECI en formato...,FECI 2005,Reporte del FECI para el 2005,http://tabulario.s3.amazonaws.com/Data/Superin...
4,Data Bancaria,Superintendencia de Bancos,"csv, feci, data bancaria",Reportes de FECI,Incluye todos los reportes del FECI en formato...,FECI 2006,Reporte del FECI para el 2006,http://tabulario.s3.amazonaws.com/Data/Superin...
5,Data Bancaria,Superintendencia de Bancos,"csv, feci, data bancaria",Reportes de FECI,Incluye todos los reportes del FECI en formato...,FECI 2007,Reporte del FECI para el 2007,http://tabulario.s3.amazonaws.com/Data/Superin...
6,Data Bancaria,Superintendencia de Bancos,"csv, feci, data bancaria",Reportes de FECI,Incluye todos los reportes del FECI en formato...,FECI 2008,Reporte del FECI para el 2008,http://tabulario.s3.amazonaws.com/Data/Superin...
7,Data Bancaria,Superintendencia de Bancos,"csv, feci, data bancaria",Reportes de FECI,Incluye todos los reportes del FECI en formato...,FECI 2009,Reporte del FECI para el 2009,http://tabulario.s3.amazonaws.com/Data/Superin...
8,Data Bancaria,Superintendencia de Bancos,"csv, feci, data bancaria",Reportes de FECI,Incluye todos los reportes del FECI en formato...,FECI 2010,Reporte del FECI para el 2010,http://tabulario.s3.amazonaws.com/Data/Superin...
9,Data Bancaria,Superintendencia de Bancos,"csv, feci, data bancaria",Reportes de FECI,Incluye todos los reportes del FECI en formato...,FECI 2011,Reporte del FECI para el 2011,http://tabulario.s3.amazonaws.com/Data/Superin...


Iterates over all rows in the Catalog checking to see if the Organizations and Groups exist on the server.

If not, attempts to create the missing Group or Organization, and then reindexes so our data can reflect the new record.

In [114]:
for ix, row in catalog.iterrows():
    try:    
        if row['Group (Topics)'] not in groups_nameindex.keys():
            groupname = unidecode.unidecode(row['Group (Topics)']).lower().replace(' ','_')
            groupname = re.sub('[()\.,!@#$]', '', groupname)
            print('Missing group:', row['Group (Topics)'])
            print('creating group...')
            print(groupname, row['Group (Topics)'])
            ckan.action.group_create(name=groupname, title=row['Group (Topics)'])
            reindex_groups()
        else:
            pass
    except ValueError:
        print(row)
        raise ValueError

In [115]:
for ix, row in catalog.iterrows():
    if row['Organization'] not in orgs_nameindex.keys():
        orgname = unidecode.unidecode(row['Organization']).lower().replace(' ','_')
        orgname = re.sub('[()\.,!@#$]', '', orgname)
        print('Missing organization', row['Organization'], orgname)
        print('Creating Organization...')
        ckan.action.organization_create(name=orgname, title = row['Organization'])
        reindex_orgs()
        

## Example record of a datapackage.json
Thought you might be interested


```{"groups":[],"isopen":true,"license_id":"cc-zero","license_title":"Creative Commons CCZero","license_url":"http://www.opendefinition.org/licenses/cc-zero","name":"2cli-bulk-test","notes":"Minimalist Import Test","num_resources":1,"num_tags":2,"owner_org":"9b203007-c03a-45bd-9916-6c84a549fce3","private":false,"resources":[{"description":"Trying to add this thing from the CLI! Lets see","format":"CSV","name":"2I'm a URL Resource","position":0,"state":"active","url":"http://tabulario.s3.amazonaws.com/Data/Presupuestos%20de%20la%20Naci%C3%B3n/Presupuesto%202015%20-%20Solicitado%2C%20Asignado%20y%20Ejecutado/2015-Table%201.csv"}],"state":"active","tags":[{"display_name":"MEF","name":"MEF"},{"display_name":"MEH","name":"MEH"}],"title":"2CLI Dataset","type":"dataset"}```

In [84]:
def get_orgID(record):
    return orgs_nameindex[record['Organization']]['id']

In [85]:
def get_orgSlug(record):
    return orgs_nameindex[record['Organization']]['name']

In [120]:
def getGroupinfo(record):
    global i
    groupInfo = {}
    try:
        bigGroup = groups_nameindex[record['Group (Topics)']]
    except Exception as e:
        print (i, record)
        raise(e)
    groupInfo['id']=bigGroup['id']
    groupInfo['display_name'] = bigGroup['display_name']
    groupInfo['name'] = bigGroup['name']
    groupInfo['title'] = bigGroup['title']
    
    return [groupInfo]

In [118]:
def create_baseDataset(record, name):
    dataset = {}
    # Basics
    dataset['isopen'] = True
    dataset['license_id'] = 'cc-zero'
    dataset['license_title'] = 'Creative Commons CCZero'
    dataset['license_url'] = 'http://opendefinition.org/licenses/cc-zero/'
    dataset['private'] = False
    dataset['state'] = 'active'
    dataset['type'] = 'dataset'
    dataset['resources'] = []
    dataset['tags'] = []
    
    # Record-Based
    dataset['groups'] = getGroupinfo(record)
    dataset['name'] = name
    
    try:
        dataset['notes'] = record['Dataset Description']
    except:
        dataset['notes'] = ""
    
    
    dataset['owner_org'] = get_orgID(record)
    dataset['title'] = record['Dataset Name']
    
    
    return dataset
    

# IF YOU NEED TO REGENERATE THE OUTPUT AFTER MAKING CHANGES TO YOUR IMPORT CSV, RE-RUN THE FOLLOWING CELL

In [150]:
i = 0
cat_len = len(catalog)
jsonl = ""

allDatasets = OrderedDict()

for ix, record in catalog.iterrows():
    
    #record = catalog.loc(i)
    #print(i, ix)
    #print(record)
    
    # Generate Dataset name slug
    try:
        name = unidecode.unidecode(record['Dataset Name']).lower().replace(' ','_')
        name = re.sub('[()\.,!@#$]', '', name)
        name = get_orgSlug(record)+'--'+name
    except Exception as e:
        print("skipping record", record, i)
        pass
    
    
    # Test to see if we've seen this dataset before, and if not: create it
    if not name in allDatasets:
        allDatasets[name] = create_baseDataset(record, name)
    
    currentDataset = allDatasets[name]
    
    
    # Add new Resource
    resource = {}
    resource['name'] = unidecode.unidecode(record['Resource Name'])
    resource['position'] = len(currentDataset['resources'])
    resource['description'] = record['Resource Description']
    resource['state'] = 'active'
    resource['url'] = record['Resource URL']
    # Add to Dataset
    currentDataset['resources'].append(resource)
    
    # get tags
    for t in record['Tags (comma-separated)'].split(','):
        if t is not "":
            thisTag = {'name':t.strip()}
            if thisTag not in currentDataset['tags']:
                currentDataset['tags'].append(thisTag)
    
    
# When complete the first time
for d in allDatasets:
    currentDataset = allDatasets[d]
    # Add counts for resources + tags
    currentDataset['num_resources'] = len(currentDataset['resources'])
    currentDataset['num_tags'] = len(currentDataset['tags'])
    
    jsonl += json.dumps(currentDataset)+'\n'

with open(OUTPUT_DATASETS_JSONL, 'w') as f:
    f.write(jsonl)

In [151]:
!ckanapi load datasets -I {OUTPUT_DATASETS_JSONL} -p 1 --upload-resources --remote={CKAN_URL} --apikey={CKAN_APIKEY}

1 [2] --- update None "superintendencia-de-bancos--reportes_de_feci"
2 [3] 1.07s create None "superintendencia-de-bancos--balances_de_situacion_del_centro_bancario_internacional"
3 [4] 2.14s create None "ministerio-de-economia-y-finanzas--viaticos_del_mef"
4 [5] 6.49s create None "autoridad_de_defensa_del_consumidor_acodeco--canasta_basica"
5 [6] 0.36s create None "asamblea-nacional--ejecucion_presupuestaria"
6 [7] 0.38s create None "asamblea-nacional--planilla"
7 [8] 1.43s create None "asamblea-nacional--diputados"
8 [9] 1.20s create None "asamblea-nacional--estadistica_de_leyes"
9 [10] 3.10s create None "ministerio_de_desarrollo_social_mides--padrino_empresario"
10 [11] 0.51s create None "ministerio_de_gobierno_mingob--ejecucion_presupuestaria"
11 [12] 2.56s create None "ministerio_de_comercio_e_industrias--importacion_y_exportacion"
12 [13] 2.05s create None "empresa_de_transmision_electrica_sa--estadistica_de_capacidad_y_generacion"
13 [14] 1.57s create None "empresa_de_transmision

# THEN RESUBMIT ALL TASKS AGAIN TO THE DATAPUSHER

Remember this part requires SSH and running ckan paster commands.

1. Connect to your CKAN Server over SSH
2. Consider using tmux for these instructions
3. Send all your assets to the DataPusher - this might take a while. Try not to disconnect your computer from the internet, or be smart and do this all in tmux like a pro. If this asks for a "Yes/No [Y/n]", say Y
4. Create all missing views
5. Reindex the search engine that powers most dataset display
6. Disconnect from your server because that's way too much power.


Run this (or its equivelent) from your command line. This works for our [docker-compose CKAN setup](https://github.com/browninstitute/ckan-dockercompose-nuevanacion-tabulario), but different CKANs are set up really differently.
```
ssh -i tabulario-ckan.pem ubuntu@data.nuevanacion.com
tmux #Optional
docker exec -it ckan paster datapusher submit_all  -c /etc/ckan/default/production.ini
docker exec -it ckan paster views create -y -c /etc/ckan/default/production.ini
docker exec -t ckan paster search-index rebuild -c /etc/ckan/default/production.ini
exit
exit #Now it's safe to disconnect
```

![](https://media.giphy.com/media/3ohzdLQUbKEu47o9Ww/giphy.gif)

But seriously, Open Knowledge International (formerly Open Knowledge Foundation): **WHY ON EARTH IS THIS SO HARD?! Please streamline your getting started documentation to cover setting up CKAN in a real production environment and bootstrapping the loading of datasets. This should NOT have taken weeks for me to set up and import and you are absolutely failing the open data community by not streamlining the process of standing up a production CKAN and batch loading data not already described in a datapackage.json.**


ps

friends [dont](https://gist.github.com/tmcw/7923069) [let](http://sciencecommons.org/resources/readingroom/comments-on-odbl/) [friends](https://discuss.okfn.org/t/maintenance-and-future-of-the-open-data-commons-licences/4460/10) [use](http://geohipster.com/2015/04/27/gary-gale-dear-osm-its-time-to-get-your-finger-out/) ODBL / Open Data Commons Licenses. 